# Current News Scraper Development

<br>

### Imports

In [1]:
import pandas as pd

import requests
from bs4 import BeautifulSoup

from IPython.core.display import JSON

### <br>

### Portals

Current News also aggregate articles on solar and storage which they share through an API

In [2]:
url = 'https://www.solarpowerportal.co.uk/json'

r = requests.get(url)

r

<Response [200]>

In [8]:
r_json = r.json()

assert len(r_json) == 1, 'There should only be one list of articles' # If I start hitting this should add flatten_list
articles = r_json[0]['entries']

renames = {
    'path': 'article_url',
    'image': 'image_url'
}

for article in articles:
    for old_name, new_name in renames.items():
        value = article.pop(old_name)
        article.update( {new_name: value})

JSON(articles)

<IPython.core.display.JSON object>